In [25]:
# !pip install faiss-gpu # For CUDA 7.5+ Supported GPU's.
# OR
# !pip install faiss-cpu # For CPU Installation

import os
import openai
from dotenv import load_dotenv, find_dotenv

In [26]:
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

In [27]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

### Simple Chain

In [28]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI(verbose=True, temperature=0.0)
output_parser = StrOutputParser()

In [29]:
chain = prompt | model | output_parser

In [30]:
chain.invoke({"topic": "winter"})

'Why did the snowman bring a broom to the party?\n\nBecause he wanted to sweep everyone off their feet!'

## More complex chain

In [31]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

In [33]:
vectorstore = FAISS.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

In [34]:
retriever.get_relevant_documents(query="Where did Harrison work?")

[Document(page_content='harrison worked at kensho'),
 Document(page_content='bears like to eat honey')]

In [35]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [36]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'))])

In [37]:
from langchain.schema.runnable import RunnableMap

chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [38]:
chain.invoke({"question": "What do bears like?"})

'Bears like to eat honey.'

### Fallback

In [40]:
from langchain.llms import OpenAI
import json

In [41]:

simple_model = OpenAI(temperature=0.0, max_tokens=1000, model="gpt-3.5-turbo-instruct")
simple_chain = simple_model | json.loads

In [42]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

In [44]:
# This is expected to fail
simple_chain.invoke(challenge)

JSONDecodeError: Extra data: line 9 column 1 (char 125)

In [46]:
fallback_chain = model | StrOutputParser() | json.loads
fallback_chain.invoke(challenge)

{'poem1': {'title': 'Whispers of the Night',
  'author': 'Emily Anderson',
  'first_line': 'In the stillness of dark, whispers rise unseen'},
 'poem2': {'title': 'Dancing Leaves',
  'author': 'Michael Carter',
  'first_line': "Autumn's embrace, leaves waltz in the breeze"},
 'poem3': {'title': "Aurora's Song",
  'author': 'Sophia Rivera',
  'first_line': 'Crimson ribbons across the night sky unfurl'}}

In [47]:
final_chain = simple_chain.with_fallbacks([fallback_chain])

In [48]:
final_chain.invoke(challenge)

{'poem1': {'title': 'Whispers of the Wind',
  'author': 'Emily Dickinson',
  'first_line': 'The wind whispers secrets through the trees.'},
 'poem2': {'title': 'Dancing in Moonlight',
  'author': 'Robert Frost',
  'first_line': "Under the moon's enchanting glow, we dance."},
 'poem3': {'title': 'Colors of the Soul',
  'author': 'Maya Angelou',
  'first_line': 'In every hue, the soul finds its rhythm.'}}